In [4]:
import gc
import os
import talib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

import plotly as py
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

from os import getenv
from dotenv import load_dotenv

from strategy import long_short
from config import binance_client, prices_collection
from helper import vwap, check_decimals, current_position

load_dotenv()

client = binance_client()
prices = prices_collection()

In [8]:
last_date = prices.find().sort('date', -1).limit(1).next()['date']
print('Finding the last record in our DB')

new_data_query = client.get_historical_klines('ETHUSDT', '5m', str(last_date), limit=100)
print('Getting new data from Binance API (5m interval)')

new_data = pd.DataFrame(new_data_query[:-1], columns=['Open time', 'open', 'high', 'low', 'close', 'vol', 'Close time', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore'])
new_data['date'] = pd.to_datetime(new_data['Close time'], unit='ms')
cols_to_drop = ['Open time', 'Close time', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore']
new_data.drop(cols_to_drop, axis=1, inplace=True)

cols = ['open', 'high', 'low', 'close', 'vol']
new_data[cols] = new_data[cols].apply(pd.to_numeric, errors='ignore', axis=1)

if len(new_data) > 0:
    print('Inserting new data into DB')
    prices_collection().insert_many(new_data.to_dict('records'))

df = pd.DataFrame(list(prices.find().sort('date', -1).limit(300)))
df.sort_values('date', inplace=True)

df.set_index('date', inplace=True)
close = df['close'].values
df['rsi'] = talib.RSI(close, timeperiod=14)
macd, macdsignal, df['macdhist'] = talib.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)
df['EMA9'] = talib.EMA(df.close, 9)
df['EMA55'] = talib.EMA(df.close, 55)
df['EMA200'] = talib.EMA(df.close, 200)

df['date'] = df.index
df.reset_index(drop=True, inplace=True)
df.fillna(method='ffill', inplace=True)

print('Calculating signals')
df['signal'] = df.apply(long_short, axis=1)

symbol = 'ETHUSD'
decimal = check_decimals(symbol)

Finding the last record in our DB
Getting new data from Binance API (5m interval)
Inserting new data into DB
Calculating signals


In [9]:
signal = df['signal'].iloc[-1]
signal

'BUY'

In [11]:
with open('current_position.txt', 'w') as f:
    f.write(str(signal))

In [12]:
df.tail(20)

,_id,open,high,low,close,vol,rsi,macdhist,EMA9,EMA55,EMA200,date,signal
280,63a23076a265d709bededa2c,1215.57,1215.92,1214.95,1215.92,3.84795,54.312422,-0.000017,1214.457956,1214.824583,1207.555787,2022-12-20 21:54:59.999,None
281,63a23076a265d709bededa2d,1215.92,1215.92,1215.92,1215.92,0.00000,54.312422,0.114875,1214.750365,1214.863705,1207.639013,2022-12-20 21:59:59.999,None
282,63a231c2a265d709bededa30,1215.25,1215.29,1214.96,1215.29,1.85429,50.911540,0.144716,1214.858292,1214.878930,1207.715142,2022-12-20 22:04:59.999,None
283,63a2330ea265d709bededa33,1215.27,1215.33,1215.27,1215.27,1.59956,50.802784,0.158282,1214.940634,1214.892897,1207.790315,2022-12-20 22:09:59.999,None
284,63a2345aa265d709bededa36,1216.80,1216.80,1216.80,1216.80,0.04819,58.165219,0.259493,1215.312507,1214.961008,1207.879963,2022-12-20 22:14:59.999,BUY
285,63a23703c59dfd87b1c9eca1,1217.40,1217.41,1217.34,1217.35,0.37482,60.456169,0.347200,1215.720005,1215.046329,1207.974193,2022-12-20 22:19:59.999,BUY
286,63a23703c59dfd87b1c9eca2,1216.81,1216.81,1215.01,1215.01,2.37551,48.329795,0.236621,1215.578004,1215.045031,1208.044201,2022-12-20 22:24:59.999,None
287,63a23c69c59dfd87b1c9eca5,1214.42,1215.64,1214.37,1215.64,5.01063,51.169609,0.196555,1215.590403,1215.066280,1208.119781,2022-12-20 22:29:59.999,BUY
288,63a23c69c59dfd87b1c9eca6,1215.87,1216.35,1215.66,1215.66,1.22638,51.261189,0.163410,1215.604323,1215.087485,1208.194808,2022-12-20 22:34:59.999,BUY
289,63a23c69c59dfd87b1c9eca7,1215.67,1215.67,1215.21,1215.21,0.39576,49.032940,0.105973,1215.525458,1215.091860,1208.264611,2022-12-20 22:39:59.999,None


In [ ]:
signal = df['signal'].iloc[-1]
position = current_position()

if signal == position:
    print('No change in position')
    
else:
    with open('current_position.txt', 'w') as f:
        f.write(signal)

    if signal == 'BUY':
        print(f'Signal went from: {position} to {signal}, flipping to long at {df["close"].iloc[-1]}')
        balance = client.get_asset_balance(asset='USD')
        quantity = round((float(balance['free']) * .95), decimal)
        order = client.create_order(symbol=symbol, side=signal, type='MARKET', quantity=quantity)
        print(order)

    if signal == 'SELL':
        print(f'Signal went from: {position} to {signal}, flipping to short at {df["close"].iloc[-1]}')
        balance = client.get_asset_balance(asset='ETH')
        quantity = round((float(balance['free']) * .95), decimal)
        order = client.create_order(symbol=symbol, side=signal, type='MARKET', quantity=quantity)
        print(order)

    if signal == 'None':
        print('No signal, holding position')
        pass

In [ ]:
df.to_csv('check.csv', index=False)

In [ ]:
df.plot(y=['close', 'EMA20'], x='date', figsize=(20, 10))